In [58]:
import spacy

#nlp = spacy.blank("ko")   # blank Korean pipeline
nlp = spacy.load("ko_core_news_lg")


In [59]:
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

In [60]:
def ent(text, substring, label):
    start = text.find(substring)
    assert start != -1, f"'{substring}' not found in text"
    end = start + len(substring)
    return (start, end, label)


In [61]:
TRAIN_DATA = [
    (
        "김민수 매니저가 담당하는 베타는 내부 민감 프로젝트이다.",
        {"entities": [(0, 3, "PROJECT_MANAGER"), (14, 16, "PROJECT_NAME")]}
    ),
    (
        "데이터 통합 프로젝트는 2025년 3월 1일부터 2025년 3월 1일까지 진행된다.",
        {"entities": [(0, 4, "PROJECT_NAME")]}
    ),
    (
        "프로젝트 관리는 매우 중요한 업무입니다.",
        {"entities": []}
    ),
    (
        "델타 프로젝트는 2027년 2월 28일부터 2026년 12월 31일까지 진행된다.",
        {"entities": [(0, 2, "PROJECT_NAME")]}
    ),
    (
        "우리는 좋은 팀 환경을 만들기 위해 노력하고 있습니다.",
        {"entities": []}
    ),
    (
        "전략기획팀 주관으로 진행되는 클라우드 전환의 책임자는 정하늘이다.",
        {"entities": [(0, 5, "ORG_INTERNAL"), (29, 32, "PROJECT_MANAGER")]}
    ),
    (
        "보안개발팀 주관으로 진행되는 감마의 책임자는 최유진이다.",
        {"entities": [(0, 5, "ORG_INTERNAL"), (25, 28, "PROJECT_MANAGER"), (16, 18, "PROJECT_NAME")]}
    ),
    (
        "회사의 목표는 고객 만족을 최우선으로 하는 것입니다.",
        {"entities": []}
    ),
    (
        "프로젝트 차세대 보안는 총 예산 22억 원으로 정하늘 PM이 담당한다.",
        {"entities": [(26, 29, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 데이터 통합는 총 예산 22억 원으로 정하늘 PM이 담당한다.",
        {"entities": [(26, 29, "PROJECT_MANAGER")]}
    ),
    (
        "이 문서는 공개 가능한 일반 정보입니다.",
        {"entities": []}
    ),
    (
        "델타 프로젝트의 예산은 5억 원으로 책정되었다.",
        {"entities": [(0, 2, "PROJECT_NAME")]}
    ),
    (
        "프로젝트 코드명 레드존는 내부 문서에서만 사용된다.",
        {"entities": [(9, 14, "PROJECT_TERM")]}
    ),
    (
        "업무 효율성을 높이기 위해 새로운 도구를 도입했습니다.",
        {"entities": []}
    ),
    (
        "감마 프로젝트는 2026년 1월 1일부터 2027년 2월 28일까지 진행된다.",
        {"entities": [(0, 2, "PROJECT_NAME")]}
    ),
    (
        "프로젝트 차세대 보안는 총 예산 12억 원으로 김민수 PM이 담당한다.",
        {"entities": [(26, 29, "PROJECT_MANAGER")]}
    ),
    (
        "팀 회의는 매주 목요일에 진행됩니다.",
        {"entities": []}
    ),
    (
        "R&D혁신팀 주관으로 진행되는 델타의 책임자는 윤도현이다.",
        {"entities": [(0, 6, "ORG_INTERNAL"), (26, 29, "PROJECT_MANAGER"), (17, 19, "PROJECT_NAME")]}
    ),
    (
        "프로젝트 코드명 내부등급A는 내부 문서에서만 사용된다.",
        {"entities": [(9, 14, "PROJECT_TERM")]}
    ),
    (
        "고객 만족도는 우리의 최고 우선순위입니다.",
        {"entities": []}
    ),
    (
        "프로젝트 AI 분석는 총 예산 22억 원으로 윤도현 PM이 담당한다.",
        {"entities": [(25, 28, "PROJECT_MANAGER")]}
    ),
    (
        "베타 프로젝트의 예산은 22억 원으로 책정되었다.",
        {"entities": [(0, 2, "PROJECT_NAME")]}
    ),
    (
        "안녕하세요. 어떻게 도와드릴까요?",
        {"entities": []}
    ),
    (
        "송지훈 매니커가 담당하는 내부 감사는 내부 민감 프로젝트이다.",
        {"entities": [(0, 3, "PROJECT_MANAGER"), (14, 18, "PROJECT_NAME")]}
    ),
    (
        "이 기능은 모든 사용자에게 공개되어 있습니다.",
        {"entities": []}
    ),
    (
        "오메가 프로젝트의 책임자는 이준혁이다.",
        {"entities": [(0, 3, "PROJECT_NAME"), (14, 17, "PROJECT_MANAGER")]}
    ),
    (
        "당사는 2025년부터 새로운 서비스를 제공하고 있습니다.",
        {"entities": []}
    ),
    (
        "마케팅팀 주관으로 진행되는 브랜드 혁신의 담당자는 박미진이다.",
        {"entities": [(0, 5, "ORG_INTERNAL"), (26, 29, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 스마트팩토리는 총 예산 50억 원으로 책정되었다.",
        {"entities": [(5, 12, "PROJECT_NAME")]}
    ),
    (
        "팀원들과의 협력이 성공의 열쇠입니다.",
        {"entities": []}
    ),
    (
        "알파 프로젝트는 2026년 6월부터 2027년 5월까지 진행될 예정이다.",
        {"entities": [(0, 2, "PROJECT_NAME")]}
    ),
    (
        "회의실은 3층에 위치하고 있습니다.",
        {"entities": []}
    ),
    (
        "재무팀 주관으로 진행되는 시스템 통합의 책임자는 서동욱이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (26, 29, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 클라우드 마이그레이션는 총 예산 30억 원으로 진행된다.",
        {"entities": [(5, 16, "PROJECT_NAME")]}
    ),
    (
        "업무 프로세스 개선을 위한 피드백을 환영합니다.",
        {"entities": []}
    ),
    (
        "감시팀에서 진행하는 보안강화 프로젝트의 PM은 홍길동이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (24, 27, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 코드명 블루싸이는 극비 프로젝트이다.",
        {"entities": [(9, 13, "PROJECT_TERM")]}
    ),
    (
        "우리 회사는 여러 나라에서 사업을 하고 있습니다.",
        {"entities": []}
    ),
    (
        "장비팀 주관으로 진행되는 하드웨어 업그레이드의 책임자는 강민철이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (27, 30, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 모바일 앱 개발는 총 예산 15억 원으로 진행 중이다.",
        {"entities": [(5, 13, "PROJECT_NAME")]}
    ),
    (
        "이는 공개 문서이므로 누구나 접근할 수 있습니다.",
        {"entities": []}
    ),
    (
        "데이터팀에서 담당하는 AI 학습 모델의 매니저는 최우성이다.",
        {"entities": [(0, 4, "ORG_INTERNAL"), (25, 28, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 코드명 자금조달은 외부 공개 금지이다.",
        {"entities": [(9, 13, "PROJECT_TERM")]}
    ),
    (
        "새로운 정책에 대해 더 알고 싶으신가요?",
        {"entities": []}
    ),
    (
        "엔지니어링팀 주관으로 진행되는 인프라 현대화의 담당자는 이영준이다.",
        {"entities": [(0, 6, "ORG_INTERNAL"), (27, 30, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 블록체인 파일럿은 총 예산 8억 원으로 책정되었다.",
        {"entities": [(5, 13, "PROJECT_NAME")]}
    ),
    (
        "문제 해결을 위한 새로운 방법론을 제시했습니다.",
        {"entities": []}
    ),
    (
        "운영팀에서 추진하는 자동화 프로젝트의 책임자는 한정수이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (21, 24, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 코드명 프론티어는 특별 관리 대상이다.",
        {"entities": [(9, 13, "PROJECT_TERM")]}
    ),
    (
        "직원들의 역량 개발이 중요한 과제입니다.",
        {"entities": []}
    ),
    (
        "영업팀 주관으로 진행되는 시장 확대의 매니저는 조금희이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (25, 28, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 HR 시스템은 총 예산 25억 원으로 진행된다.",
        {"entities": [(5, 10, "PROJECT_NAME")]}
    ),
    (
        "고객 피드백은 우리의 개선 방향을 결정합니다.",
        {"entities": []}
    ),
    (
        "법무팀에서 담당하는 규정 정비 프로젝트의 PM은 김도엽이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (24, 27, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 코드명 실크로드는 국제 협력 프로젝트이다.",
        {"entities": [(9, 13, "PROJECT_TERM")]}
    ),
    (
        "회사의 이익 창출이 모든 활동의 목표입니다.",
        {"entities": []}
    ),
    (
        "콘텐츠팀 주관으로 진행되는 디지털 마케팅의 담당자는 박준호이다.",
        {"entities": [(0, 4, "ORG_INTERNAL"), (26, 29, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 보안감시는 총 예산 18억 원으로 책정되었다.",
        {"entities": [(5, 10, "PROJECT_NAME")]}
    ),
    (
        "새로운 기술 도입으로 업무 효율이 향상되었습니다.",
        {"entities": []}
    ),
    (
        "인사팀에서 진행하는 복지 확대 프로젝트의 책임자는 이소영이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (22, 25, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 코드명 골든게이트는 제한된 접근만 허용된다.",
        {"entities": [(9, 14, "PROJECT_TERM")]}
    ),
    (
        "사회 공헌 활동도 우리의 경영 철학 중 하나입니다.",
        {"entities": []}
    ),
    (
        "품질팀 주관으로 진행되는 서비스 개선의 매니저는 정재운이다.",
        {"entities": [(0, 3, "ORG_INTERNAL"), (25, 28, "PROJECT_MANAGER")]}
    ),
    (
        "프로젝트 공급망 최적화는 총 예산 35억 원으로 진행 중이다.",
        {"entities": [(5, 12, "PROJECT_NAME")]}
    ),
    (
        "사무직과 현장직의 협력이 성공적으로 이루어지고 있습니다.",
        {"entities": []}
    )
]


In [62]:
from spacy.training import offsets_to_biluo_tags

for text, ann in TRAIN_DATA:
    doc = nlp.make_doc(text)
    tags = offsets_to_biluo_tags(doc, ann["entities"])
    if "-" in tags:
        print("❌ MISALIGNED:", text)
        print(tags)


❌ MISALIGNED: 김민수 매니저가 담당하는 베타는 내부 민감 프로젝트이다.
['U-PROJECT_MANAGER', 'O', 'O', '-', 'O', 'O', 'O', 'O']
❌ MISALIGNED: 데이터 통합 프로젝트는 2025년 3월 1일부터 2025년 3월 1일까지 진행된다.
['-', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
❌ MISALIGNED: 전략기획팀 주관으로 진행되는 클라우드 전환의 책임자는 정하늘이다.
['U-ORG_INTERNAL', 'O', 'O', 'O', 'O', 'O', '-', 'O']
❌ MISALIGNED: 보안개발팀 주관으로 진행되는 감마의 책임자는 최유진이다.
['U-ORG_INTERNAL', 'O', 'O', '-', 'O', '-', 'O']
❌ MISALIGNED: 프로젝트 코드명 레드존는 내부 문서에서만 사용된다.
['O', 'O', '-', 'O', 'O', 'O', 'O']
❌ MISALIGNED: R&D혁신팀 주관으로 진행되는 델타의 책임자는 윤도현이다.
['U-ORG_INTERNAL', 'O', 'O', '-', 'O', '-', 'O']
❌ MISALIGNED: 프로젝트 코드명 내부등급A는 내부 문서에서만 사용된다.
['O', 'O', '-', 'O', 'O', 'O', 'O']
❌ MISALIGNED: 송지훈 매니커가 담당하는 내부 감사는 내부 민감 프로젝트이다.
['U-PROJECT_MANAGER', 'O', 'O', '-', '-', 'O', 'O', 'O', 'O']
❌ MISALIGNED: 오메가 프로젝트의 책임자는 이준혁이다.
['U-PROJECT_NAME', 'O', 'O', '-', 'O']
❌ MISALIGNED: 마케팅팀 주관으로 진행되는 브랜드 혁신의 담당자는 박미진이다.
['-', 'O', 'O', 'O', 'O', '-', '-', 'O']
❌ MISALIGNED: 재무팀 주관으로 진행되는 시스템 통합의 책임자는 서동욱이다.


c:\Users\user\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "김민수 매니저가 담당하는 베타는 내부 민감 프로젝트이다." with entities "[(0, 3, 'PROJECT_MANAGER'), (14, 16, 'PROJECT_NAME...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\user\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "데이터 통합 프로젝트는 2025년 3월 1일부터 2025년 3월 1일까지 진행된다." with entities "[(0, 4, 'PROJECT_NAME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\user\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "전략기획팀 주관으로 진행되는 클라우드 전환의 책임자는 정

In [63]:
#add label from my data 
for _, annotations in TRAIN_DATA:
    for start, end, label in annotations["entities"]:
        ner.add_label(label)


In [64]:
#print all new enities labels
for label in ner.labels:
    print(label)

DT
LC
OG
ORG_INTERNAL
PROJECT_MANAGER
PROJECT_NAME
PROJECT_TERM
PS
QT
TI


In [65]:
from spacy.training import Example

examples = []
for text, ann in TRAIN_DATA:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, ann)
    examples.append(example)


In [66]:
#optimizer = nlp.initialize(lambda: examples) for blank ko pipeline

In [67]:
optimizer = nlp.resume_training() #for already trained ko pipeline

In [68]:
import random

for epoch in range(20):
    random.shuffle(examples)
    losses = {}

    nlp.update(
        examples,
        sgd=optimizer,
        drop=0.4,
        losses=losses
    )

    print(f"Epoch {epoch+1}, Losses: {losses}")



Epoch 1, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 89.31621692720951}
Epoch 2, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 66.02741247817175}
Epoch 3, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 53.029148820831324}
Epoch 4, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 48.415861405843316}
Epoch 5, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 41.69399503966633}
Epoch 6, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 37.59469249015092}
Epoch 7, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 0.0, 'lemmatizer': 0.0, 'ner': 35.14973804241481}
Epoch 8, Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'morphologizer': 0.0, 'parser': 

In [69]:
def make_entity(text, substring, label):
    start = text.find(substring)
    if start == -1:
        return None
    end = start + len(substring)
    return (start, end, label)


In [70]:
text = "김하늘 매니저가 담당하는 프로젝트 알파는 총 예산 100억 원이다."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

김하늘 PROJECT_MANAGER


In [71]:
doc = nlp("김철수는 연세대학교에 다니며 이메일은 abc99@naver.com이다.")

print([(ent.text, ent.label_) for ent in doc.ents])


[('김철수는', 'PROJECT_MANAGER')]


In [72]:
from spacy.training import offsets_to_biluo_tags

text = "김민수 매니저가 담당하는 베타는 내부 민감 프로젝트이다."
entities = [(14, 19, "PROJECT_NAME")]

doc = nlp.make_doc(text)
print(offsets_to_biluo_tags(doc, entities))


['O', 'O', 'O', '-', '-', 'O', 'O', 'O']
